In [1]:
from warnings import filterwarnings

filterwarnings("ignore")
import os

import pandas as pd
import numpy as np
import polars as pl
from gc import collect
from tqdm.notebook import tqdm
from scipy.optimize import minimize, Bounds
from numba import njit

In [2]:
train = pd.read_csv(f"data/train.csv", index_col="date_id")
N = len(train)

In [3]:
train = pl.read_csv(f"data/train.csv")
N = len(train)

In [4]:
train

date_id,D1,D2,D3,D4,D5,D6,D7,D8,D9,E1,E10,E11,E12,E13,E14,E15,E16,E17,E18,E19,E2,E20,E3,E4,E5,E6,E7,E8,E9,I1,I2,I3,I4,I5,I6,I7,…,P13,P2,P3,P4,P5,P6,P7,P8,P9,S1,S10,S11,S12,S2,S3,S4,S5,S6,S7,S8,S9,V1,V10,V11,V12,V13,V2,V3,V4,V5,V6,V7,V8,V9,forward_returns,risk_free_rate,market_forward_excess_returns
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,…,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,f64
0,0,0,0,1,1,0,0,0,1,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,-0.002421,0.000301,-0.003038
1,0,0,0,1,1,0,0,0,1,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,-0.008495,0.000303,-0.009114
2,0,0,0,1,0,0,0,0,1,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,-0.009624,0.000301,-0.010243
3,0,0,0,1,0,0,0,0,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.004662,0.000299,0.004046
4,0,0,0,1,0,0,0,0,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,-0.011686,0.000299,-0.012301
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
9016,0,0,0,1,0,0,0,0,0,"""1.4931173355878""","""0.174272486772487""","""0.00231481481481481""","""0.00231481481481481""","""0.00231481481481481""","""0.00231481481481481""","""0.928240740740741""","""-0.38178125864808""","""-0.499284930236135""","""0.0872407728267281""","""-0.689459715799012""","""1.41412844031523""","""1.46910780893469""","""1.84805508043407""","""0.0357142857142857""","""0.321428571428571""","""0.0244708994708995""","""0.109229322951801""","""-0.327450224813603""","""0.999669312169312""","""0.360780423280423""","""0.732658971704731""","""0.0750661375661376""","""0.457010582010582""","""0.525397366182665""","""0.703703703703704""","""0.345238095238095""",…,"""0.0975529100529101""","""-1.52434259213177""","""0.623015873015873""","""0.0671296296296296""","""-0.810821388675249""","""-0.0690250721925946""","""1.86803145876213""","""1.8771103538522""","""0.0443121693121693""","""0.362123981171164""","""0.957671957671958""","""0.886243386243386""","""0.263975515911582""","""-0.0740926316615758""","""0.156729820901314""","""0.69973544973545""","""-0.373982425528818""","""0.37037037037037""","""0.948412698412698""","""0.808137369564275""","""0.0257936507936508""","""0.345899470899471""","""-0.346977743153752""","""0.122685185185185""","""0.35978835978836""","""-0.251129288381404""","""0.541005291005291""","""0.208994708994709""","""0.48478835978836""","""0.717308297979935""","""0.677248677248677""","""-0.327455331344421""","""0.083994708994

In [ ]:
MIN_INVESTMENT, MAX_INVESTMENT = 0, 2

rfr, fwd = train["risk_free_rate"].to_numpy(), train["forward_returns"].to_numpy()


class ParticipantVisibleError(Exception):
    pass


@njit
def ScoreMetric(submission, rfr, fwd _):
    pos = submission
    if pos.max() > MAX_INVESTMENT:
        raise ParticipantVisibleError(f"Max {pos.max()} > {MAX_INVESTMENT}")
    if pos.min() < MIN_INVESTMENT:
        raise ParticipantVisibleError(f"Min {pos.min()} < {MIN_INVESTMENT}")

    strat_ret = rfr * (1 - pos) + pos * fwd

    excess_ret = strat_ret - rfr
    mean_excess = (1 + excess_ret).prod() ** (1 / N) - 1
    std = strat_ret.std()
    if std == 0:
        raise ZeroDivisionError

    sharpe = mean_excess / std * np.sqrt(252)
    strat_vol = std * np.sqrt(252) * 100

    market_vol = fwd.std() * np.sqrt(252) * 100
    market_mean = (1 + fwd - rfr).prod() ** (1 / N) - 1

    vol_penalty = 1 + max(0, strat_vol / market_vol - 1.2) if market_vol > 0 else 0
    return_penalty = 1 + ((max(0, (market_mean - mean_excess) * 100 * 252)) ** 2) / 100

    return min(sharpe / (vol_penalty * return_penalty), 1e6)

In [6]:
def fun(x):
    """
    Optimization logic adapted from Hull Tactical Kaggle discussion #608349.
    """
    ...

    x.clip(0, 2)
    return -ScoreMetric(x, "")


x0 = np.full(len(train), 0.05)
res = minimize(fun, x0, method="Powell", bounds=Bounds(lb=0, ub=2), tol=1e-8)

print(res)

opt_preds = res.x

 message: Maximum number of function evaluations has been exceeded.
 success: False
  status: 1
     fun: -16.53644561743369
       x: [ 5.402e-09  5.402e-09 ...  1.672e-01  1.432e-01]
     nit: 31
   direc: [[ 1.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
           [ 0.000e+00  1.000e+00 ...  0.000e+00  0.000e+00]
           ...
           [ 0.000e+00  0.000e+00 ...  1.000e+00  0.000e+00]
           [ 0.000e+00  0.000e+00 ...  0.000e+00  1.000e+00]]
    nfev: 9021000


In [7]:
pd.DataFrame({"prediction": opt_preds[-180:]})

,prediction
0,2.000198e-01
1,2.000000e+00
2,5.401778e-09
3,1.275095e-01
4,5.406927e-09
...,...
175,5.406927e-09
176,5.401778e-09
177,2.321223e-01
178,1.672283e-01


In [8]:
train[-180:]

date_id,D1,D2,D3,D4,D5,D6,D7,D8,D9,E1,E10,E11,E12,E13,E14,E15,E16,E17,E18,E19,E2,E20,E3,E4,E5,E6,E7,E8,E9,I1,I2,I3,I4,I5,I6,I7,…,P13,P2,P3,P4,P5,P6,P7,P8,P9,S1,S10,S11,S12,S2,S3,S4,S5,S6,S7,S8,S9,V1,V10,V11,V12,V13,V2,V3,V4,V5,V6,V7,V8,V9,forward_returns,risk_free_rate,market_forward_excess_returns
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,…,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,f64
8841,0,0,0,1,0,0,0,0,0,"""2.0154466355633""","""0.389219576719577""","""0.0595238095238095""","""0.0343915343915344""","""0.0310846560846561""","""0.00529100529100529""","""0.828042328042328""","""0.28674676826844""","""-0.24026756587415""","""-0.242672885356905""","""0.559318783160083""","""1.0936233108282""","""1.20297723578946""","""1.54539314965004""","""0.0145502645502645""","""0.127314814814815""","""0.00562169312169312""","""0.0208808732795314""","""-0.324214539739531""","""0.975198412698413""","""0.526455026455027""","""1.10944804159701""","""0.273148148148148""","""0.564814814814815""","""0.853747369693238""","""0.472222222222222""","""0.529100529100529""",…,"""0.345899470899471""","""-1.43723644986491""","""0.789351851851852""","""0.647486772486772""","""0.19667213283189""","""-0.184309012297845""","""-1.26090110517594""","""2.2055077659526""","""0.0376984126984127""","""-1.07684179771063""","""0.704365079365079""","""0.953703703703704""","""0.569315503900396""","""-0.279738732811623""","""-0.498425804144831""","""0.98478835978836""","""-0.171782270775772""","""0.530092592592593""","""0.201388888888889""","""0.250810162315464""","""0.49537037037037""","""0.334656084656085""","""-0.651545915948012""","""0.257275132275132""","""0.0740740740740741""","""-0.530430263205069""","""0.475529100529101""","""0.154761904761905""","""0.312830687830688""","""2.08049969716496""","""0.351851851851852""","""-0.481273792581522""","""0.100859788359788""","""-0.539076572604003""",0.006791,0.000168,0.006312
8842,0,0,0,1,0,0,0,0,0,"""2.01136214225578""","""0.388888888888889""","""0.0591931216931217""","""0.0340608465608466""","""0.0307539682539683""","""0.00496031746031746""","""0.828373015873016""","""0.28702019443388""","""-0.239612898372235""","""-0.242920756201707""","""-0.781369684968353""","""1.1525769664201""","""1.20134821481125""","""1.61111987242467""","""0.0142195767195767""","""0.126984126984127""","""0.00529100529100529""","""0.0208472422646957""","""-0.324181979202372""","""0.975529100529101""","""0.517857142857143""","""1.15475725549761""","""0.26521164021164""","""0.564484126984127""","""0.852957911245475""","""0.472883597883598""","""0.528439153439153""",…,"""0.503968253968254""","""-1.44129160935278""","""0.12202380952381""","""0.393187830687831""","""-0.0348041616052411""","""0.213670746487049""","""0.799155693892945""","""2.22006291889402""","""0.0330687830687831""","""-1.08418436352655""","""0.654100529100529""","""0.949074074074074""","""0.773007580932077""","""-0.211873869684544""","""-0.681726228802856""","""0.947751322751323""","""-0.226418738443182""","""0.407407407407407""","""0.201388888888889""","""0.249929357934702""","""0.312830687830688""","""0.327380952380952""","""-0.703930629122569""","""0.178902116402116""","""0.183862433862434""","""-0.586802964526804""","""0.445767195767196""","""0.197751322751323""","""0.312830687830688""","""2.29427834737869""","""0.343915343915344""","""-0.573188320793073""","""0.0975529100529101""","""-0.596066497279078""",0.000761,0.000168,0.000281
8843,0,0,0,1,0,0,0,0,0,"""2.00729612730355""","""0.388558201058201""","""0.0588624338624339""","""0.0337301587301587""","""0.0304232804232804""","""0.00462962962962963""","""0.828703703703704""","""0.287293702839884""","""-0.238958089850549""","""-0.243168687674453""","""-0.729788888969428""","""1.15253532844112""","""1.19973132543488""","""1.60999

In [ ]:
import numpy as np
import pandas as pd
import pandas.api.types

MIN_INVESTMENT = 0
MAX_INVESTMENT = 2


class ParticipantVisibleError(Exception):
    pass


def score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str) -> float:
    """
    Calculates a custom evaluation metric (volatility-adjusted Sharpe ratio).

    This metric penalizes strategies that take on significantly more volatility
    than the underlying market.

    Returns:
        float: The calculated adjusted Sharpe ratio.
    """

    if not pandas.api.types.is_numeric_dtype(submission["prediction"]):
        raise ParticipantVisibleError("Predictions must be numeric")

    solution = solution.copy()
    solution["position"] = submission["prediction"]

    display(solution)

    if solution["position"].max() > MAX_INVESTMENT:
        raise ParticipantVisibleError(f"Position of {solution['position'].max()} exceeds maximum of {MAX_INVESTMENT}")
    if solution["position"].min() < MIN_INVESTMENT:
        raise ParticipantVisibleError(f"Position of {solution['position'].min()} below minimum of {MIN_INVESTMENT}")

    solution["strategy_returns"] = solution["risk_free_rate"] * (1 - solution["position"]) + solution["position"] * solution["forward_returns"]

    # Calculate strategy's Sharpe ratio
    strategy_excess_returns = solution["strategy_returns"] - solution["risk_free_rate"]
    strategy_excess_cumulative = (1 + strategy_excess_returns).prod()
    strategy_mean_excess_return = (strategy_excess_cumulative) ** (1 / len(solution)) - 1
    strategy_std = solution["strategy_returns"].std()

    trading_days_per_yr = 252
    if strategy_std == 0:
        raise ParticipantVisibleError("Division by zero, strategy std is zero")
    sharpe = strategy_mean_excess_return / strategy_std * np.sqrt(trading_days_per_yr)
    strategy_volatility = float(strategy_std * np.sqrt(trading_days_per_yr) * 100)

    # Calculate market return and volatility
    market_excess_returns = solution["forward_returns"] - solution["risk_free_rate"]
    market_excess_cumulative = (1 + market_excess_returns).prod()
    market_mean_excess_return = (market_excess_cumulative) ** (1 / len(solution)) - 1
    market_std = solution["forward_returns"].std()

    market_volatility = float(market_std * np.sqrt(trading_days_per_yr) * 100)

    if market_volatility == 0:
        raise ParticipantVisibleError("Division by zero, market std is zero")

    # Calculate the volatility penalty
    excess_vol = max(0, strategy_volatility / market_volatility - 1.2) if market_volatility > 0 else 0
    vol_penalty = 1 + excess_vol

    # Calculate the return penalty
    return_gap = max(
        0,
        (market_mean_excess_return - strategy_mean_excess_return) * 100 * trading_days_per_yr,
    )
    return_penalty = 1 + (return_gap**2) / 100

    # Adjust the Sharpe ratio by the volatility and return penalty
    print(sharpe, (vol_penalty, return_penalty))
    adjusted_sharpe = sharpe / (vol_penalty * return_penalty)
    return min(float(adjusted_sharpe), 1_000_000)


In [ ]:
train.to_pandas().set_index("date_id")

,D1,D2,D3,D4,D5,D6,D7,D8,D9,E1,...,V3,V4,V5,V6,V7,V8,V9,forward_returns,risk_free_rate,market_forward_excess_returns
date_id,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,1,1,0,0,0,1,None,...,None,None,None,None,None,None,None,-0.002421,0.000301,-0.003038
1,0,0,0,1,1,0,0,0,1,None,...,None,None,None,None,None,None,None,-0.008495,0.000303,-0.009114
2,0,0,0,1,0,0,0,0,1,None,...,None,None,None,None,None,None,None,-0.009624,0.000301,-0.010243
3,0,0,0,1,0,0,0,0,0,None,...,None,None,None,None,None,None,None,0.004662,0.000299,0.004046
4,0,0,0,1,0,0,0,0,0,None,...,None,None,None,None,None,None,None,-0.011686,0.000299,-0.012301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9016,0,0,0,1,0,0,0,0,0,1.4931173355878,...,0.208994708994709,0.48478835978836,0.717308297979935,0.677248677248677,-0.327455331344421,0.083994708994709,-0.380451915677545,-0.000015,0.000151,-0.000477
9017,0,0,0,1,0,0,0,0,0,1.49088907791661,...,0.082010582010582,0.482804232804233,1.00102798773345,0.596560846560847,-0.372978740645921,0.0942460317460317,-0.427354855211809,-0.005199,0.000150,-0.005661
9018,0,0,0,1,0,-1,0,0,0,1.48866687658991,...,0.334656084656085,0.486772486772487,0.894501972948187,0.656746031746032,-0.282023556404646,0.0906084656084656,-0.381337108171407,0.005930,0.000150,0.005467


In [13]:
score(
    solution=train.to_pandas().set_index("date_id")[-180:],
    submission=pd.DataFrame({"prediction": opt_preds[-180:]}, index=train.to_pandas().set_index("date_id")[-180:].index),
    row_id_column_name="",
)

,D1,D2,D3,D4,D5,D6,D7,D8,D9,E1,...,V4,V5,V6,V7,V8,V9,forward_returns,risk_free_rate,market_forward_excess_returns,position
date_id,,,,,,,,,,,,,,,,,,,,,
8841,0,0,0,1,0,0,0,0,0,2.0154466355633,...,0.312830687830688,2.08049969716496,0.351851851851852,-0.481273792581522,0.100859788359788,-0.539076572604003,0.006791,0.000168,0.006312,2.000198e-01
8842,0,0,0,1,0,0,0,0,0,2.01136214225578,...,0.312830687830688,2.29427834737869,0.343915343915344,-0.573188320793073,0.0975529100529101,-0.596066497279078,0.000761,0.000168,0.000281,2.000000e+00
8843,0,0,0,1,0,0,0,0,0,2.00729612730355,...,0.375,2.31970785582403,0.401455026455026,-0.546824927445363,0.128637566137566,-0.58914360470971,-0.003221,0.000168,-0.003700,5.401778e-09
8844,0,0,1,1,0,0,0,0,0,2.00324844164363,...,0.369708994708995,2.45069687122196,0.51984126984127,-0.563137251053616,0.118716931216931,-0.584982266669676,0.010558,0.000168,0.010078,1.275095e-01
8845,0,0,0,1,0,0,0,0,0,1.99921893789731,...,0.371031746031746,2.24548187694532,0.543650793650794,-0.662506691052428,0.114748677248677,-0.651858751698863,-0.000049,0.000167,-0.000528,5.406927e-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9016,0,0,0,1,0,0,0,0,0,1.4931173355878,...,0.48478835978836,0.717308297979935,0.677248677248677,-0.327455331344421,0.083994708994709,-0.380451915677545,-0.000015,0.000151,-0.000477,5.406927e-09
9017,0,0,0,1,0,0,0,0,0,1.49088907791661,...,0.482804232804233,1.00102798773345,0.596560846560847,-0.372978740645921,0.0942460317460317,-0.427354855211809,-0.005199,0.000150,-0.005661,5.401778e-09
9018,0,0,0,1,0,-1,0,0,0,1.48866687658991,...,0.486772486772487,0.894501972948187,0.656746031746032,-0.282023556404646,0.0906084656084656,-0.381337108171407,0.005930,0.000150,0.005467,2.321223e-01


16.95270521151928 (1, 1.0)


16.95270521151928

In [19]:
pl.DataFrame({"y":opt_preds}).insert_column(0, train.select('date_id').to_series()).write_csv("train_labels.csv")